In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import pandas as pd
movies=pd.read_csv("movies_cleaned.csv")

In [5]:
movies

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,tagged_description
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",0 Two imprisoned men bond over a number of yea...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",1 An organized crime dynasty's aging patriarch...
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",2 When the menace known as the Joker wreaks ha...
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",3 The early life and career of Vito Corleone i...
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",4 A jury holdout attempts to prevent a miscarr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN,995 A young New York socialite becomes interes...
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN,996 Sprawling epic covering the life of a Texa...
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000","997 In Hawaii in 1941, a private is cruelly pu..."
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN,998 Several survivors of a torpedoed merchant ...


In [6]:
movies["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

In [14]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [8]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.base import Embeddings
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from typing import List


In [9]:
class TFIDFEmbeddings(Embeddings):
    def __init__(self):
        self.vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
        self.fitted = False
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        tfidf_matrix = self.vectorizer.fit_transform(texts)
        self.fitted = True
        embeddings = []
        for i in range(tfidf_matrix.shape[0]):
            embedding = tfidf_matrix[i].toarray()[0].tolist()
            embeddings.append(embedding)
        return embeddings
    
    def embed_query(self, text: str) -> List[float]:
        if not self.fitted:
            raise ValueError("Call embed_documents first!")
        query_vector = self.vectorizer.transform([text])
        return query_vector.toarray()[0].tolist()

In [10]:
embeddings = TFIDFEmbeddings()
vectordb = Chroma.from_documents(documents=documents, embedding=embeddings)

print(" Vector database created successfully!")

 Vector database created successfully!


In [11]:
query = "A movie about adventures and fantasy"
docs = vectordb.similarity_search(query, k=10)

print(f"\nFound {len(docs)} documents for: '{query}'")
for i, doc in enumerate(docs[:5], 1):  # Show first 5
    print(f"\n--- Result {i} ---")
    print(doc.page_content[:200] + "...")



Found 10 documents for: 'A movie about adventures and fantasy'

--- Result 1 ---
150 In the Falangist Spain of 1944, the bookish young stepdaughter of a sadistic army officer escapes into an eerie but captivating fantasy world.
151 The toys are mistakenly delivered to a day-care c...

--- Result 2 ---
590 A mistaken delivery in Mumbai's famously efficient lunchbox delivery system connects a young housewife to an older man in the dusk of his life as they build a fantasy world together through notes ...

--- Result 3 ---
169 In this luminous tale set in the area around Sarajevo and in Italy, Perhan, an engaging young Romany (gypsy) with telekinetic powers, is seduced by the quick-cash world of petty crime, which threa...

--- Result 4 ---
659 A successful lawman's plans to retire anonymously in Tombstone, Arizona are disrupted by the kind of outlaws he was famous for eliminating.
660 In the summer of 1962, a new kid in town is taken un...

--- Result 5 ---
968 An ordinary man frustrated

In [ ]:
# SMART APPROACH: Store movie index in metadata
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import re

# Load the raw file
raw_documents = TextLoader("tagged_description.txt").load()

# Split into lines first to process each movie separately
lines = raw_documents[0].page_content.strip().split('\n')

# Create documents with proper metadata
documents_with_metadata = []

for line in lines:
    if line.strip():  # Skip empty lines
        # Extract the movie index from the beginning of the line
        # This handles various formats: "123 Movie Title", 123 Movie Title, etc.
        match = re.match(r'["\']?(\d+)["\']?\s+(.+)', line.strip())
        
        if match:
            movie_index = int(match.group(1))
            movie_content = match.group(2)
            
            # Create document with movie index in metadata
            doc = Document(
                page_content=movie_content,  # Just the description, no index
                metadata={
                    'movie_index': movie_index,
                    'source': 'tagged_description.txt'
                }
            )
            documents_with_metadata.append(doc)
            print(f"✅ Processed movie {movie_index}: {movie_content[:50]}...")

print(f"\n📊 Created {len(documents_with_metadata)} documents with metadata")

# Now split if needed (optional - since each line is probably one movie)
text_splitter = CharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=0, 
    separator="\n"
)

# Split documents but preserve metadata
final_documents = []
for doc in documents_with_metadata:
    if len(doc.page_content) > 500:
        # Split long descriptions but keep the same movie_index
        chunks = text_splitter.split_documents([doc])
        for chunk in chunks:
            chunk.metadata['movie_index'] = doc.metadata['movie_index']  # Preserve movie index
            final_documents.append(chunk)
    else:
        final_documents.append(doc)

print(f"📚 Final documents after splitting: {len(final_documents)}")

# Create vector database
from langchain.vectorstores import Chroma
from langchain.embeddings.base import Embeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import List

class TFIDFEmbeddings(Embeddings):
    def __init__(self):
        self.vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
        self.fitted = False
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        tfidf_matrix = self.vectorizer.fit_transform(texts)
        self.fitted = True
        embeddings = []
        for i in range(tfidf_matrix.shape[0]):
            embedding = tfidf_matrix[i].toarray()[0].tolist()
            embeddings.append(embedding)
        return embeddings
    
    def embed_query(self, text: str) -> List[float]:
        query_vector = self.vectorizer.transform([text])
        return query_vector.toarray()[0].tolist()

embeddings = TFIDFEmbeddings()
vectordb = Chroma.from_documents(documents=final_documents, embedding=embeddings)

print("✅ Vector database created with metadata!")

# NOW THE MAGIC - No more parsing needed!
query = "A book to teach children about nature"
docs = vectordb.similarity_search(query, k=10)

print(f"\n🔍 Query: '{query}'")
print(f"📚 Found {len(docs)} relevant chunks")

# Extract movie indexes directly from metadata - NO PARSING!
recommended_indexes = [doc.metadata['movie_index'] for doc in docs]

print(f"🎯 Recommended movie indexes: {recommended_indexes}")

# Get movie titles
recommended_movies = movies[movies.index.isin(recommended_indexes)]["Series_Title"]

print(f"\n🎬 RECOMMENDED MOVIES:")
print("=" * 40)
for idx, title in recommended_movies.items():
    print(f"{idx}: {title}")

# Show which chunks led to each recommendation
print(f"\n📝 CHUNK DETAILS:")
print("=" * 50)
for i, doc in enumerate(docs, 1):
    movie_idx = doc.metadata['movie_index']
    movie_title = movies.loc[movie_idx, "Series_Title"] if movie_idx in movies.index else "Unknown"
    print(f"\n--- Result {i} ---")
    print(f"🎬 Movie: {movie_title} (Index: {movie_idx})")
    print(f"📄 Chunk: {doc.page_content[:150]}...")

print("\n" + "="*60)
print("✅ SUCCESS! Clean solution with metadata:")
print("="*60)
print("# Now your query is simply:")
print("docs = vectordb.similarity_search(query, k=10)")
print("recommended_indexes = [doc.metadata['movie_index'] for doc in docs]")
print("movies[movies.index.isin(recommended_indexes)]['Series_Title']")

✅ Processed movie 0: Two imprisoned men bond over a number of years, fi...
✅ Processed movie 1: An organized crime dynasty's aging patriarch trans...
✅ Processed movie 2: When the menace known as the Joker wreaks havoc an...
✅ Processed movie 3: The early life and career of Vito Corleone in 1920...
✅ Processed movie 4: A jury holdout attempts to prevent a miscarriage o...
✅ Processed movie 5: Gandalf and Aragorn lead the World of Men against ...
✅ Processed movie 6: The lives of two mob hitmen, a boxer, a gangster a...
✅ Processed movie 7: In German-occupied Poland during World War II, ind...
✅ Processed movie 8: A thief who steals corporate secrets through the u...
✅ Processed movie 9: An insomniac office worker and a devil-may-care so...
✅ Processed movie 10: A meek Hobbit from the Shire and eight companions ...
✅ Processed movie 11: The presidencies of Kennedy and Johnson, the event...
✅ Processed movie 12: A bounty hunting scam joins two men in an uneasy a...
✅ Processed movie 13: 

In [20]:
print("DEBUGGING METADATA:")
print("="*50)

print("Number of docs returned:", len(docs))
print("\nFirst few docs metadata:")
for i, doc in enumerate(docs[:3]):
    print(f"\nDoc {i+1}:")
    print(f"  Metadata keys: {list(doc.metadata.keys())}")
    print(f"  Metadata content: {doc.metadata}")
    print(f"  Content preview: {doc.page_content[:100]}...")

print("\n" + "="*60)
print("POSSIBLE FIXES:")
print("="*60)

# Function to extract movie index from metadata or fallback from content
def get_movie_index_fallback(doc):
    """Extract movie index from metadata or fallback using regex from content"""
    if 'movie_index' in doc.metadata:
        return doc.metadata['movie_index']
    
    import re
    content = doc.page_content
    match = re.match(r'["\']?(\d+)["\']?\s', content)
    if match:
        return int(match.group(1))
    
    numbers = re.findall(r'\b\d+\b', content)
    if numbers:
        return int(numbers[0])
    
    return None

# Attempt to extract indexes using the fallback method
print("Attempting fallback index extraction:")
recommended_indexes = []
for i, doc in enumerate(docs):
    index = get_movie_index_fallback(doc)
    if index is not None:
        recommended_indexes.append(index)
        print(f"  Doc {i+1}: Extracted index {index}")
    else:
        print(f"  Doc {i+1}: Failed to extract index")

print(f"\nExtracted indexes: {recommended_indexes}")

if recommended_indexes:
    try:
        recommended_movies = movies[movies.index.isin(recommended_indexes)]["Series_Title"]
        print("\nRecommended Movies:")
        for idx, title in recommended_movies.items():
            print(f"  {idx}: {title}")
    except Exception as e:
        print(f"Error retrieving movie titles: {e}")

print("\n" + "="*60)
print("RE-CREATE VECTOR DATABASE WITH METADATA:")
print("="*60)

# Re-loading the document file and parsing each line to extract index and content
from langchain.document_loaders import TextLoader
from langchain.schema import Document
import re

print("Loading and processing document lines...")

raw_documents = TextLoader("tagged_description.txt").load()
raw_content = raw_documents[0].page_content

print("Sample content from raw file:")
print(repr(raw_content[:200]))

lines = raw_content.strip().split('\n')
print(f"\nTotal lines found: {len(lines)}")

documents_with_metadata = []
failed_lines = []

for line_num, line in enumerate(lines, 1):
    line = line.strip()
    if not line:
        continue

    patterns = [
        r'^["\']?(\d+)["\']?\s+(.+)$',
        r'^(\d+)\s*[\-:]\s*(.+)$',
        r'^(\d+)\s+(.+)$',
    ]

    extracted = False
    for pattern in patterns:
        match = re.match(pattern, line)
        if match:
            try:
                movie_index = int(match.group(1))
                movie_content = match.group(2).strip()

                if movie_content:
                    doc = Document(
                        page_content=movie_content,
                        metadata={'movie_index': movie_index, 'source': 'tagged_description.txt'}
                    )
                    documents_with_metadata.append(doc)
                    print(f"Line {line_num}: Parsed Index {movie_index} - {movie_content[:50]}...")
                    extracted = True
                    break
            except ValueError:
                continue

    if not extracted:
        failed_lines.append((line_num, line[:100]))
        print(f"Line {line_num}: Failed to parse - {line[:50]}...")

print(f"\nDocuments successfully processed: {len(documents_with_metadata)}")
print(f"Lines failed to process: {len(failed_lines)}")

# Create vector database if valid documents were parsed
if documents_with_metadata:
    from langchain.vectorstores import Chroma
    from langchain.embeddings.base import Embeddings
    from sklearn.feature_extraction.text import TfidfVectorizer
    from typing import List

    # Custom TF-IDF based embedding implementation
    class TFIDFEmbeddings(Embeddings):
        def __init__(self):
            self.vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
            self.fitted = False
        
        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            tfidf_matrix = self.vectorizer.fit_transform(texts)
            self.fitted = True
            embeddings = []
            for i in range(tfidf_matrix.shape[0]):
                embedding = tfidf_matrix[i].toarray()[0].tolist()
                embeddings.append(embedding)
            return embeddings
        
        def embed_query(self, text: str) -> List[float]:
            query_vector = self.vectorizer.transform([text])
            return query_vector.toarray()[0].tolist()

    print("\nCreating new vector store...")
    embeddings = TFIDFEmbeddings()
    vectordb_new = Chroma.from_documents(documents=documents_with_metadata, embedding=embeddings)

    # Run similarity search using test query
    query = "A book to teach children about nature"
    docs_new = vectordb_new.similarity_search(query, k=10)

    print(f"\nQuery used: '{query}'")
    print(f"Documents returned: {len(docs_new)}")

    recommended_indexes_new = [doc.metadata['movie_index'] for doc in docs_new]
    print(f"Recommended indexes: {recommended_indexes_new}")

    if recommended_indexes_new:
        recommended_movies_new = movies[movies.index.isin(recommended_indexes_new)]["Series_Title"]
        print("\nRecommended Movies:")
        for idx, title in recommended_movies_new.items():
            print(f"  {idx}: {title}")

    print("\nVector database creation complete. Use 'vectordb_new' for further queries.")
else:
    print("No documents were processed successfully. Check formatting of the input file.")


DEBUGGING METADATA:
Number of docs returned: 10

First few docs metadata:

Doc 1:
  Metadata keys: ['movie_index', 'source']
  Metadata content: {'movie_index': 274, 'source': 'tagged_description.txt'}
  Content preview: Two young Swedish children experience the many comedies and tragedies of their family, the Ekdahls....

Doc 2:
  Metadata keys: ['source', 'movie_index']
  Metadata content: {'source': 'tagged_description.txt', 'movie_index': 274}
  Content preview: Two young Swedish children experience the many comedies and tragedies of their family, the Ekdahls....

Doc 3:
  Metadata keys: ['movie_index', 'source']
  Metadata content: {'movie_index': 245, 'source': 'tagged_description.txt'}
  Content preview: In order to power the city, monsters have to scare children so that they scream. However, the childr...

POSSIBLE FIXES:
Attempting fallback index extraction:
  Doc 1: Extracted index 274
  Doc 2: Extracted index 274
  Doc 3: Extracted index 245
  Doc 4: Extracted index 245
  Do

In [21]:

query = "A book to teach children about nature"
docs = vectordb_new.similarity_search(query, k=10)


recommended_indexes = [doc.metadata['movie_index'] for doc in docs]
recommended_movies = movies[movies.index.isin(recommended_indexes)]["Series_Title"]

print(" Recommended Movies:")
for idx, title in recommended_movies.items():
    print(f"{idx}: {title}")

 Recommended Movies:
245: Monsters, Inc.
274: Fanny och Alexander
429: The Sound of Music
563: His Girl Friday


In [28]:
def retrieve_semantic_recommendations(
    query: str, 
    top_k: int = 10
) -> pd.DataFrame:
    """
    Get movie recommendations based on semantic similarity search
    
    Args:
        query: Search query (e.g., "A book to teach children about nature")
        top_k: Number of recommendations to return
        
    Returns:
        DataFrame with recommended movies (filtered from original movies DataFrame)
    """
    import re
    
    def get_movie_index(doc):
        """Safely extract movie index from metadata or content"""
        # Try metadata first
        if 'movie_index' in doc.metadata:
            return doc.metadata['movie_index']
        
        # Fallback: extract from content
        content = doc.page_content
        match = re.match(r'(\d+)', content)
        return int(match.group(1)) if match else None
    
    # Search for similar movie descriptions
    recs = vectordb_new.similarity_search(query, k=50)
    
    # Extract movie indexes safely
    movie_indexes = [get_movie_index(doc) for doc in recs if get_movie_index(doc) is not None]
    
    # Filter original movies DataFrame to only recommended movies
    recommended_movies = movies[movies.index.isin(movie_indexes)].head(top_k)
    
    return recommended_movies

# Example usage:
query = "A book to teach children about nature"
recommendations = retrieve_semantic_recommendations(query, top_k=5)


In [27]:
recommendations

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,tagged_description
106,https://m.media-amazon.com/images/M/MV5BZGU2OG...,Aliens,1986,U,137 min,"Action, Adventure, Sci-Fi",8.3,Fifty-seven years after surviving an apocalypt...,84.0,James Cameron,Sigourney Weaver,Michael Biehn,Carrie Henn,Paul Reiser,652719,"85,160,248",106 Fifty-seven years after surviving an apoca...
180,https://m.media-amazon.com/images/M/MV5BNmVmYz...,To Kill a Mockingbird,1962,U,129 min,"Crime, Drama",8.2,"Atticus Finch, a lawyer in the Depression-era ...",88.0,Robert Mulligan,Gregory Peck,John Megna,Frank Overton,Rosemary Murphy,293811,NaN,"180 Atticus Finch, a lawyer in the Depression-..."
188,https://m.media-amazon.com/images/M/MV5BYWQ4ZT...,Tôkyô monogatari,1953,U,136 min,Drama,8.2,An old couple visit their children and grandch...,NaN,Yasujirô Ozu,Chishû Ryû,Chieko Higashiyama,Sô Yamamura,Setsuko Hara,53153,NaN,188 An old couple visit their children and gra...
237,https://m.media-amazon.com/images/M/MV5BMTQ1Mj...,Before Sunset,2004,R,80 min,"Drama, Romance",8.1,"Nine years after Jesse and Celine first met, t...",90.0,Richard Linklater,Ethan Hawke,Julie Delpy,Vernon Dobtcheff,Louise Lemoine Torrès,236311,"5,820,649",237 Nine years after Jesse and Celine first me...
245,https://m.media-amazon.com/images/M/MV5BMTY1NT...,"Monsters, Inc.",2001,U,92 min,"Animation, Adventure, Comedy",8.1,"In order to power the city, monsters have to s...",79.0,Pete Docter,David Silverman,Lee Unkrich,Billy Crystal,John Goodman,815505,"289,916,256","245 In order to power the city, monsters have ..."
